In [ ]:
import pandas as pd
import numpy as np
import io
import csv
from tqdm import tqdm
#!pip install catboost

pd.set_option("display.precision", 2)

## Сбор данных

In [ ]:
# Добавление транзакций в формате [ид пользователя, код типа транзакции]
data = []
path_to_inter_file = '/content/drive/MyDrive/data-fusion2022-education/data/transactions.csv'
with io.open(path_to_inter_file, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in tqdm(reader):
        data.append(row[:2])

In [ ]:
# # Словарь ид пользователя : коды всех транзакций всего уникальных пользователей 22533
# temp = {}
# for i in tqdm(range(1, len(data))): # со 2го чтобы не включать заголовок
#     if data[i][0] in temp:
#         temp[data[i][0]].append(data[i][1])
#     else:
#         temp.update({data[i][0]: [data[i][1]]})

# Словарь ид пользователя: код трнзакции: количество
temp = {}
for i in tqdm(range(1, len(data))):
  if data[i][0] in temp:
    if data[i][1] in temp[data[i][0]]:
      temp[data[i][0]][data[i][1]]=int(temp[data[i][0]][data[i][1]])+1
    else:
      temp[data[i][0]][data[i][1]] = '1'
  else:
    temp.update({data[i][0]: {data[i][1]: '1'}})

In [ ]:
# Преобразование словаря в df
df=pd.DataFrame.from_dict(temp)
df=df.transpose()
df=df.astype(str).astype(float)
df.fillna(0., inplace=True)
df.head()

In [ ]:
# Добавление транзакций в формате [ид пользователя, код валюты]
data = []
# path_to_inter_file = '/content/data/transactions.csv'
path_to_inter_file = '/content/drive/MyDrive/data-fusion2022-education/data/transactions.csv'
with io.open(path_to_inter_file, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    for row in tqdm(reader):
        indices = [0, 2]
        a = [row[index] for index in indices]
        data.append(a)

In [ ]:
# Словарь ид пользователя: код валюты: количество
temp = {}
for i in tqdm(range(1, len(data))):
  if data[i][0] in temp:
    if data[i][1] in temp[data[i][0]]:
      temp[data[i][0]][data[i][1]]=int(temp[data[i][0]][data[i][1]])+1
    else:
      temp[data[i][0]][data[i][1]] = '1'
  else:
    temp.update({data[i][0]: {data[i][1]: '1'}})

In [ ]:
# Преобразование словаря в df
df1=pd.DataFrame.from_dict(temp) # , index=
df1=df1.transpose()
df1=df1.astype(str).astype(float)
df1.fillna(0., inplace=True)
df1.head()

In [ ]:
# Объединение кодов валют евро и доллара
forex = []
for index, row in df.iterrows():
  a = ((row['50'])+(row['60']))
  forex.append(a)

df1['forex'] = forex
df1.drop(['50', '60'], axis=1, inplace=True)
df1.rename(columns={"-1": "cur_other", "48": "rub"}, inplace=True)
df1

In [ ]:
#Объединение датасетов
df2 = pd.merge(df, df1, how='left', left_index=True, right_index=True)
df2
df2.to_csv('/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv')

In [ ]:
#Дополнительные фичи, которые в дальнейшем был опущены

In [ ]:
#Фича 1 Попудярный код транзакции
df1['Top_tr'] = df1.idxmax(axis=1)
df1= df1.drop(df1.columns[0:-1], axis=1)
df1.head()


In [ ]:
#Фича 2 Количество транзакций
df1['Trans_qt']=df.sum(axis=1)

In [ ]:
#Фича 3 Менее популярная категория
df1['Low_tr'] = df.idxmin(axis=1)
df1.head()

In [ ]:
#Фича 4 Вторая популярная категория
for i, row in tqdm(df.iterrows()):
  df.at[i, df1.loc[i]['Top_tr']] = 0
df1['Top_tr_2'] = df.idxmax(axis=1)

In [ ]:
#Фича 5 Третья популярная категория
for i, row in tqdm(df.iterrows()):
  df.at[i, df1.loc[i]['Top_tr_2']] = 0
df1['Top_tr_3'] = df.idxmax(axis=1)

## Вычисление моделей

### Logit

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import  LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
# Подготовка данных для обучения
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/train.csv", index_col=0)
dftr = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)

In [ ]:
X = dftr.drop('higher_education', axis=1)
y = dftr["higher_education"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
Logit = LogisticRegression(
    C=10,
    class_weight={0: 1.6, 1: 1.},
    intercept_scaling=18,
    solver = 'liblinear',
    )
Logit.fit(x_train, y_train)
#Logit.fit(X, Y)

In [ ]:
# Hyperparameter tuning
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
folds = 5

pipeline = Pipeline([
    ('LogisticRegression', LogisticRegression(random_state=42))
])
parameters = { 
    'LogisticRegression__C': (10, 15, 9),
    'LogisticRegression__class_weight' : ({0: 1.3, 1: 1.}, {0: 1.6, 1: 1.}, {0: 1.5, 1: 1.}),
    'LogisticRegression__intercept_scaling' : (10, 15, 9),
    'LogisticRegression__solver' : ('liblinear', 'lbfgs')
}
c_val = StratifiedKFold(n_splits=folds)

grid_search_tune = GridSearchCV(pipeline, parameters, cv=c_val, verbose=3, n_jobs=4, scoring='f1_macro')
grid_search_tune.fit(x_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

In [ ]:
y_pred=Logit.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Прогноз для отправляемых данных 
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/sample_submission.csv", index_col=0)
dfsb = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)
dfsb.drop('higher_education_proba', axis=1, inplace=True)
dfsb

In [ ]:
prediction = Logit.predict_proba(dfsb)

In [ ]:
dfsb.reset_index(inplace=True)
dfp = pd.DataFrame(prediction[:,1], columns = ['Prediction'])
dfsb = pd.merge(dfp, dfsb, how='left', left_index=True, right_index=True)
dfsb.set_index('bank', inplace=True)
dfsb = dfsb[['Prediction', '5411']]
dfsb.drop('5411', axis=1, inplace=True)
dfsb['higher_education_proba'] = dfsb['Prediction']
dfsb.drop('Prediction', axis=1, inplace=True)
dfsb.to_csv("/content/drive/MyDrive/data-fusion2022-education/data/Logit_trans.csv")

### Catboost

In [ ]:
import catboost
from catboost import *

In [ ]:
# Данные
dftr = pd.read_csv("Data_noft.csv", index_col=0)
dfn = pd.read_csv("train.csv", index_col=0)
dftr = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)

In [ ]:
X = dftr.drop(['higher_education', 'day_time_transaction_if_night'], axis=1)
y = dftr["higher_education"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [54]:
#Модель
model = CatBoostClassifier(iterations=100,
                           eval_metric='AUC',
                           learning_rate=0.1,
                           depth=3,
                          class_weights=(2, 1),
                          l2_leaf_reg= 7)
# Fit model
model.fit(X, y,
          #eval_set=(x_test, y_test),
          plot=True,
          verbose=False)
# Get predictions
preds = model.predict(x_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
# Hyperparameter tuning
model = CatBoostClassifier()

grid = {'learning_rate': [0.1],
        'depth': [3],
        'l2_leaf_reg': [7, 9, 10]}

grid_search_result = model.grid_search(grid, 
                                       X=X, 
                                       y=y, 
                                       plot=True)

In [ ]:
dftr = pd.read_csv("Data_noft.csv", index_col=0)
dfn = pd.read_csv("sample_submission.csv", index_col=0)
dfsb = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)
dfsb.drop('higher_education_proba', axis=1, inplace=True)
dfsb.drop('day_time_transaction_if_night', axis=1, inplace=True)

In [ ]:
prediction = model.predict_proba(dfsb)

In [ ]:
dfsb.reset_index(inplace=True)
dfp = pd.DataFrame(prediction[:,1], columns = ['Prediction'])
dfsb = pd.merge(dfp, dfsb, how='left', left_index=True, right_index=True)
dfsb.set_index('bank', inplace=True)
dfsb = dfsb[['Prediction', '5411']]
dfsb.drop('5411', axis=1, inplace=True)
dfsb['higher_education_proba'] = dfsb['Prediction']
dfsb.drop('Prediction', axis=1, inplace=True)
dfsb.to_csv("CB_trans2.csv")

### XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, KFold

import matplotlib.pyplot as plt

In [ ]:
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/train.csv", index_col=0)
dftr = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)

In [ ]:
X = dftr.drop(['higher_education', 'day_time_transaction_if_night'], axis=1)
y = dftr["higher_education"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
xgb.XGBClassifier().get_params()

In [ ]:
model_xgb = xgb.XGBClassifier(learning_rate=0.02, 
                              max_depth=3, 
                              n_estimators=1000, 
                              subsample=0.5, 
                              colsample_bytree=0.5,
                              eval_metric='auc',
                              verbosity=1)

eval_set = [(x_test, y_test)]

model_xgb.fit(x_train,
              y_train,
              early_stopping_rounds=10,
              eval_set=eval_set,
              verbose=True)

In [ ]:
scores = cross_val_score(xgbc, x_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())
 
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, x_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

In [ ]:
ypred = model_xgb.predict(x_test)
cr = classification_report(y_test, ypred)
print(cr) 

In [ ]:
learning_rate_list= [0.02, 0.05, 0.1]
max_depth_list = [2, 3, 5]
n_estimators_list = [1000, 2000, 3000]

params_dict = {'learning_rate': learning_rate_list,
               'max_depth' : max_depth_list,
               'n_estimators' : n_estimators_list}

num_combinations=1
for v in params_dict.values(): num_combinations *= len(v)

print(num_combinations)
params_dict

In [ ]:
def my_roc_auc_score(model, X, y): return roc_auc_score(y, model.predict_proba(X)[:,1])

model_xgb_hp = GridSearchCV(estimator=XGBClassifier(subsample=0.5,
                                                    colsample_bytree=0.25,
                                                    eval_metric='auc',
                                                    use_label_encoder=False),
                            param_grid=params_dict,
                            cv=2,
                            scoring=my_roc_auc_score,
                            return_train_score=True,
                            verbose=4)

model_xgb_hp.fit(X, y)

In [ ]:
ypred = model_xgb.predict(x_test)

In [ ]:
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/sample_submission.csv", index_col=0)
dfsb = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)
dfsb.drop('higher_education_proba', axis=1, inplace=True)
dfsb.drop('day_time_transaction_if_night', axis=1, inplace=True)

In [ ]:
prediction = model_xgb.predict_proba(dfsb)

In [ ]:
dfsb.reset_index(inplace=True)
dfp = pd.DataFrame(prediction[:,1], columns = ['Prediction'])
dfsb = pd.merge(dfp, dfsb, how='left', left_index=True, right_index=True)
dfsb.set_index('bank', inplace=True)
dfsb = dfsb[['Prediction', '5411']]
dfsb.drop('5411', axis=1, inplace=True)
dfsb['higher_education_proba'] = dfsb['Prediction']
dfsb.drop('Prediction', axis=1, inplace=True)
dfsb.to_csv("/content/drive/MyDrive/data-fusion2022-education/data/XGB.csv")

### SDG

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipelin

In [ ]:
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/train.csv", index_col=0)
dftr = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)

In [ ]:
X = dftr.drop('higher_education', axis=1)
y = dftr["higher_education"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, 
                                  alpha=1e1, 
                                  loss = 'modified_huber', 
                                  penalty = 'l2', 
                                  random_state=61, 
                                  class_weight={0: 2.1, 1: 1.0}, 
                                  fit_intercept=True,
                                  tol=1e-2,
                                  learning_rate= 'adaptive',
                                  eta0= 1))

clf.fit(x_train, y_train)

In [ ]:
score = clf.score(x_train, y_train)
print("Training score: ", score)
ypred = clf.predict(x_test)

cm = confusion_matrix(y_test, ypred)
print(cm) 

cr = classification_report(y_test, ypred)
print(cr) 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
folds = 2

pipeline = Pipeline([
    ('SGDClassifier', SGDClassifier(random_state=42))
])
parameters = {
    'SGDClassifier__loss' : ["hinge", "log"], # "squared_hinge", "modified_huber", "perceptron"
    'SGDClassifier__penalty': ('l2', 'l1'), 
    'SGDClassifier__alpha': np.linspace(9, 9),  # убавить сотку если очень долго возможно упадет если learning_rate будет не optimal. Тогда надо убрать следующий параметр alpha
    'SGDClassifier__learning_rate': ['optimal'], # тут надо не в грид серче пробовать ставить разное значение eta0 если пробовать другие параметры
    'SGDClassifier__class_weight': ({0: 2., 1: 1.}, {0: 1.8, 1: 1.})
}
c_val = StratifiedKFold(n_splits=folds)

grid_search_tune = GridSearchCV(pipeline, parameters, cv=c_val, verbose=3, n_jobs=4, scoring='f1_macro')
grid_search_tune.fit(x_train, y_train)

print("Best parameters set:")
print(grid_search_tune.best_estimator_.steps)

In [ ]:
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/sample_submission.csv", index_col=0)
dfsb = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)
dfsb.drop('higher_education_proba', axis=1, inplace=True)
dfsb

In [ ]:
prediction = clf.predict_proba(dfsb)

In [ ]:
dfsb.reset_index(inplace=True)
dfp = pd.DataFrame(prediction[:,1], columns = ['Prediction'])
dfsb = pd.merge(dfp, dfsb, how='left', left_index=True, right_index=True)
dfsb.set_index('bank', inplace=True)
dfsb = dfsb[['Prediction', '5411']]
dfsb.drop('5411', axis=1, inplace=True)
dfsb['higher_education_proba'] = dfsb['Prediction']
dfsb.drop('Prediction', axis=1, inplace=True)
dfsb.to_csv("/content/drive/MyDrive/data-fusion2022-education/data/SDG_trans.csv")

## Объединение моделей. Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/train.csv", index_col=0)
dftr = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)

In [ ]:
X = dftr.drop('higher_education', axis=1)
y = dftr["higher_education"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
clf1 = Logit = LogisticRegression(
    C=10,
    class_weight={0: 1.6, 1: 1.},
    intercept_scaling=18,
    solver = 'liblinear',
    )
clf2 = CatBoostClassifier(iterations=500,
                           eval_metric='AUC',
                           learning_rate=0.10,
                           depth=3,
                          class_weights=(0.7, 0.3),
                          l2_leaf_reg= 7)
clf3 =  make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, 
                                  alpha=1e1, 
                                  loss = 'modified_huber', 
                                  penalty = 'l2', 
                                  random_state=61, 
                                  class_weight={0: 2.1, 1: 1.0}, 
                                  fit_intercept=True,
                                  tol=1e-2,
                                  learning_rate= 'adaptive',
                                  eta0= 1))

eclf = VotingClassifier(
     estimators=[('lr', clf1), ('cb', clf2), ('sdg', clf3)],
    voting='soft')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Catboost', 'SDG', 'Ensemble']):
     scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

eclf = eclf.fit(X, y)

In [ ]:
ypred = eclf.predict(x_test)
cr = classification_report(y_test, ypred)
print(cr) 

In [ ]:
dftr = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/Data_noft.csv", index_col=0)
dfn = pd.read_csv("/content/drive/MyDrive/data-fusion2022-education/data/sample_submission.csv", index_col=0)
dfsb = pd.merge(dfn, dftr, how='left', left_index=True, right_index=True)
dfsb.drop('higher_education_proba', axis=1, inplace=True)
dfsb

In [ ]:
prediction = eclf.predict_proba(dfsb)

In [ ]:
# Финальный результат
dfsb.reset_index(inplace=True)
dfp = pd.DataFrame(prediction[:,1], columns = ['Prediction'])
dfsb = pd.merge(dfp, dfsb, how='left', left_index=True, right_index=True)
dfsb.set_index('bank', inplace=True)
dfsb = dfsb[['Prediction', '5411']]
dfsb.drop('5411', axis=1, inplace=True)
dfsb['higher_education_proba'] = dfsb['Prediction']
dfsb.drop('Prediction', axis=1, inplace=True)
dfsb.to_csv("/content/drive/MyDrive/data-fusion2022-education/data/VotingCl_trans.csv")
dfsb